## Logistic Regression on broadband availability with demographic and RUCA variables

### All residential blocks in contiguous U.S. with complete data are considered, for year 2018
assume residential blocks are those with share of number of residential addresses greater than or equal to 80%, according to the data here: https://www.census.gov/geographies/reference-files/2020/geo/2020addcountlisting.html

In [1]:
import pandas as pd
import numpy as np

import data organized by blocks:


* blocks with NumberofUniqueHocoNums = 0 are residential blocks that don't appear in FCC form 477

In [2]:
df=pd.read_csv('fcc477_2018_grouped_ruca_demographic_byblock_additional_variables_fixed.csv')
df

,BlockCode,NumberOfUniqueProviderNames,NumberOfUniqueHocoNums,MeanMaxAdDown,MeanMaxAdUp,SdMaxAdDown,SdMaxAdUp,NumberOfUniqueTechCodes,GIDBG,State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/),...,Select County,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010",NumFCCEntriesWith25/3Speed,NumFCCEntriesWith100/10Speed,AnyProviderWith25/3,AnyProviderWith100/10
0,10010201001000,3,3,592.800000,27.256,530.777920,25.056957,5,10010201001,1001020100,...,Autauga County,1.0,1.0,1912.0,3.787641,504.799727,3,3,1,1
1,10010201001001,1,1,940.000000,35.000,NaN,NaN,1,10010201001,1001020100,...,Autauga County,1.0,1.0,1912.0,3.787641,504.799727,1,1,1,1
2,10010201001002,2,2,317.333333,12.008,539.245151,19.911656,3,10010201001,1001020100,...,Autauga County,1.0,1.0,1912.0,3.787641,504.799727,1,1,1,1
3,10010201001003,2,2,319.333333,12.008,537.521472,19.911656,3,10010201001,1001020100,...,Autauga County,1.0,1.0,1912.0,3.787641,504.799727,1,1,1,1
4,10010201001004,1,1,940.000000,35.000,NaN,NaN,1,10010201001,1001020100,...,Autauga County,1.0,1.0,1912.0,3.787641,504.799727,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8427797,410699601002983,0,0,0.000000,0.000,0.000000,0.000000,0,410699601002,41069960100,...,Wheeler County,10.0,10.0,1441.0,1714.749032,0.840356,0,0,0,0
8427798,410699601002984,0,0,0.000000,0.000,0.000000,0.000000,0,410699601002,41069960100,...,Wheeler County,10.0,10.0,1441.0,1714.749032,0.840356,0,0,0,0
8427799,410699601002986,0,0,0.000000,0.000,0.000000,0.000000,0,410699601002,41069960100,...,Wheeler County,10.0,10.0,1441.0,1714.749032,0.840356,0,0,0,0
8427800,410699601002990,0,0,0.000000,0.000,0.000000,0.000000,0,410699601002,41069960100,...,Wheeler County,10.0,10.0,1441.0,1714.749032,0.840356,0,0,0,0


In [3]:
np.unique(df['Select State']),len(np.unique(df['Select State']))

(array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
        'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
        'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
        'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
        'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object),
 52)

Filter out areas not in contiguous US:

In [4]:
df_conus=df[(df['Select State']!='AK')& (df['Select State'] !='PR') &(df['Select State'] !='HI')]
np.unique(df_conus['Select State']),len(np.unique(df_conus['Select State']))

(array(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'IA',
        'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN',
        'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY',
        'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object),
 49)

In [5]:
df_conus.columns

Index(['BlockCode', 'NumberOfUniqueProviderNames', 'NumberOfUniqueHocoNums',
       'MeanMaxAdDown', 'MeanMaxAdUp', 'SdMaxAdDown', 'SdMaxAdUp',
       'NumberOfUniqueTechCodes', 'GIDBG',
       'State-County-Tract FIPS Code (lookup by address at http://www.ffiec.gov/Geocode/)',
       'Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
       'pct_URBAN_CLUSTER_POP_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'avg_Tot_Prns_in_HHD_ACS_14_18', 'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_NH_SOR_alone_ACS_14_18', 'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18', 'pct_PUB_ASST_INC_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Buil

extract numerical columns, and reorder columns so that potential dependent variables are first (left), independent variables are after:

In [6]:
df_conus_num=df_conus[['NumberOfUniqueProviderNames','NumberOfUniqueHocoNums',\
                       'NumberOfUniqueTechCodes','NumFCCEntriesWith25/3Speed','NumFCCEntriesWith100/10Speed',\
                       'AnyProviderWith25/3','AnyProviderWith100/10',
                       'MeanMaxAdDown','MeanMaxAdUp','Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
       'pct_URBAN_CLUSTER_POP_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'avg_Tot_Prns_in_HHD_ACS_14_18', 'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_NH_SOR_alone_ACS_14_18', 'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18', 'pct_PUB_ASST_INC_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18','Primary RUCA Code 2010','Secondary RUCA Code, 2010 (see errata)', 'Tract Population, 2010',
       'Land Area (square miles), 2010',
       'Population Density (per square mile), 2010']]
df_conus_num

,NumberOfUniqueProviderNames,NumberOfUniqueHocoNums,NumberOfUniqueTechCodes,NumFCCEntriesWith25/3Speed,NumFCCEntriesWith100/10Speed,AnyProviderWith25/3,AnyProviderWith100/10,MeanMaxAdDown,MeanMaxAdUp,Tot_Population_ACS_14_18,...,pct_Prs_Blw_Pov_Lev_ACS_14_18,pct_PUB_ASST_INC_ACS_14_18,pct_Diff_HU_1yr_Ago_ACS_14_18,pct_Recent_Built_HU_ACS_14_18,avg_Agg_House_Value_ACS_14_18,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,3,3,5,3,3,1,1,592.800000,27.256,636.0,...,16.51,0.00,3.46,0.00,71265.0,1.0,1.0,1912.0,3.787641,504.799727
1,1,1,1,1,1,1,1,940.000000,35.000,636.0,...,16.51,0.00,3.46,0.00,71265.0,1.0,1.0,1912.0,3.787641,504.799727
2,2,2,3,1,1,1,1,317.333333,12.008,636.0,...,16.51,0.00,3.46,0.00,71265.0,1.0,1.0,1912.0,3.787641,504.799727
3,2,2,3,1,1,1,1,319.333333,12.008,636.0,...,16.51,0.00,3.46,0.00,71265.0,1.0,1.0,1912.0,3.787641,504.799727
4,1,1,1,1,1,1,1,940.000000,35.000,636.0,...,16.51,0.00,3.46,0.00,71265.0,1.0,1.0,1912.0,3.787641,504.799727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8427797,0,0,0,0,0,0,0,0.000000,0.000,756.0,...,20.62,3.38,11.73,0.67,226963.0,10.0,10.0,1441.0,1714.749032,0.840356
8427798,0,0,0,0,0,0,0,0.000000,0.000,756.0,...,20.62,3.38,11.73,0.67,226963.0,10.0,10.0,1441.0,1714.749032,0.840356
8427799,0,0,0,0,0,0,0,0.000000,0.000,756.0,...,20.62,3.38,11.73,0.67,226963.0,10.0,10.0,1441.0,1714.749032,0.840356
8427800,0,0,0,0,0,0,0,0.000000,0.000,756.0,...,20.62,3.38,11.73,0.67,226963.0,10.0,10.0,1441.0,1714.749032,0.840356


**Explanation of variables:**
* Broadband availability variables taken from FCC form 477: 
>'NumberOfUniqueProviderNames','NumberOfUniqueHocoNums',
'NumberOfUniqueTechCodes','NumFCCEntriesWith25/3Speed',
'NumFCCEntriesWith100/10Speed','AnyProviderWith25/3',
'AnyProviderWith100/10','MeanMaxAdDown','MeanMaxAdUp'
    
    * grouping by block code and applying aggregate functions (nunique, count, sum, etc.)
    
    
* Demographic variables by block group taken from the Census planning database (https://www.census.gov/topics/research/guidance/planning-databases.html): 

>'Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
'pct_URBAN_CLUSTER_POP_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'avg_Tot_Prns_in_HHD_ACS_14_18', 'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_NH_SOR_alone_ACS_14_18', 'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18', 'pct_PUB_ASST_INC_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18'
       
       
* RUCA variables by tract:
>'Primary RUCA Code 2010','Secondary RUCA Code, 2010 (see errata)', 
'Tract Population, 2010','Land Area (square miles), 2010','Population Density (per square mile), 2010'

### Logistic regression on availability variables

In [7]:
import statsmodels.api as sm

#### first using all independent variables, without removing multicollinear ones (we'll see it's a bad model):
* for the binary dependent variable **AnyProviderWith25/3** (1 if a block has at least 1 provider providing MaxAdDown >= 25 and MaxAdUp >= 3, according to FCC form 477, 0 otherwise)

In [8]:
y1=df_conus_num['AnyProviderWith25/3']
x=df_conus_num[['Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
       'pct_URBAN_CLUSTER_POP_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'avg_Tot_Prns_in_HHD_ACS_14_18', 'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_NH_SOR_alone_ACS_14_18', 'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18', 'pct_PUB_ASST_INC_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18','Primary RUCA Code 2010','Secondary RUCA Code, 2010 (see errata)', 'Tract Population, 2010',
       'Land Area (square miles), 2010',
       'Population Density (per square mile), 2010']]

Scale each independent variable column so that mean = 0 and standard deviation = 1, useful for logistic regression and showing the relative impact of different variables:

In [9]:
# scale  https://towardsai.net/p/data-science/how-when-and-why-should-you-normalize-standardize-rescale-your-data-3f083def38ff
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
x = scaler.fit_transform(x)
print(x.mean(axis=0))
print(x.std(axis=0))

[ 7.14510725e-18  9.41190616e-16  3.26602307e-16 -1.28418303e-15
  3.57271725e-15 -2.52533180e-17 -7.24192072e-16  2.02893774e-15
  3.26165964e-17  8.46777023e-18  2.95267466e-16 -1.27827878e-16
  2.95594723e-16 -8.67448745e-16  2.69495992e-16  1.56548754e-15
  2.70022330e-15 -1.17812455e-16 -4.39205924e-16  4.95794083e-17
  5.62199946e-16 -3.29438533e-16 -2.48442470e-17  1.12030918e-16
 -7.83889171e-16 -1.07312966e-16 -3.46892230e-16 -7.71944297e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


Re-add the feature names for readability:

In [10]:
x=pd.DataFrame(x,columns =['Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
       'pct_URBAN_CLUSTER_POP_CEN_2010', 'pct_RURAL_POP_CEN_2010',
       'avg_Tot_Prns_in_HHD_ACS_14_18', 'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_NH_SOR_alone_ACS_14_18', 'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18', 'pct_PUB_ASST_INC_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18','Primary RUCA Code 2010','Secondary RUCA Code, 2010 (see errata)', 'Tract Population, 2010',
       'Land Area (square miles), 2010',
       'Population Density (per square mile), 2010'] )
x

,Tot_Population_ACS_14_18,pct_URBANIZED_AREA_POP_CEN_2010,pct_URBAN_CLUSTER_POP_CEN_2010,pct_RURAL_POP_CEN_2010,avg_Tot_Prns_in_HHD_ACS_14_18,pct_Vacant_Units_ACS_14_18,pct_Hispanic_ACS_14_18,pct_NH_White_alone_ACS_14_18,pct_NH_Blk_alone_ACS_14_18,pct_NH_AIAN_alone_ACS_14_18,...,pct_Prs_Blw_Pov_Lev_ACS_14_18,pct_PUB_ASST_INC_ACS_14_18,pct_Diff_HU_1yr_Ago_ACS_14_18,pct_Recent_Built_HU_ACS_14_18,avg_Agg_House_Value_ACS_14_18,Primary RUCA Code 2010,"Secondary RUCA Code, 2010 (see errata)","Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,-0.714121,0.738754,-0.42865,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,...,0.245343,-0.683823,-1.017627,-0.436151,-0.550049,-0.818349,-0.818568,-1.221334,-0.337507,-0.300638
1,-0.714121,0.738754,-0.42865,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,...,0.245343,-0.683823,-1.017627,-0.436151,-0.550049,-0.818349,-0.818568,-1.221334,-0.337507,-0.300638
2,-0.714121,0.738754,-0.42865,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,...,0.245343,-0.683823,-1.017627,-0.436151,-0.550049,-0.818349,-0.818568,-1.221334,-0.337507,-0.300638
3,-0.714121,0.738754,-0.42865,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,...,0.245343,-0.683823,-1.017627,-0.436151,-0.550049,-0.818349,-0.818568,-1.221334,-0.337507,-0.300638
4,-0.714121,0.738754,-0.42865,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,...,0.245343,-0.683823,-1.017627,-0.436151,-0.550049,-0.818349,-0.818568,-1.221334,-0.337507,-0.300638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337435,-0.628434,-0.844677,-0.42865,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,...,0.605055,0.364676,-0.071693,-0.229007,0.292540,1.914737,1.909261,-1.439816,3.222259,-0.436382
8337436,-0.628434,-0.844677,-0.42865,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,...,0.605055,0.364676,-0.071693,-0.229007,0.292540,1.914737,1.909261,-1.439816,3.222259,-0.436382
8337437,-0.628434,-0.844677,-0.42865,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,...,0.605055,0.364676,-0.071693,-0.229007,0.292540,1.914737,1.909261,-1.439816,3.222259,-0.436382
8337438,-0.628434,-0.844677,-0.42865,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,...,0.605055,0.364676,-0.071693,-0.229007,0.292540,1.914737,1.909261,-1.439816,3.222259,-0.436382


Reset index on dependent variable so they match with x's index:

In [11]:
y1.reset_index(drop=True,inplace=True)
y1

0          1
1          1
2          1
3          1
4          1
          ..
8337435    0
8337436    0
8337437    0
8337438    0
8337439    0
Name: AnyProviderWith25/3, Length: 8337440, dtype: int64

Fit logistic model with lasso penalty: 

In [12]:
model = sm.Logit(y1, x) # all features
result = model.fit_regularized()
result.summary() # some large p values

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6306061501753609
            Iterations: 132
            Function evaluations: 132
            Gradient evaluations: 132


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     AnyProviderWith25/3   No. Observations:              8337440
Model:                           Logit   Df Residuals:                  8337412
Method:                            MLE   Df Model:                           27
Date:                 Wed, 19 May 2021   Pseudo R-squ.:                 -0.2687
Time:                         14:49:02   Log-Likelihood:            -5.2576e+06
converged:                        True   LL-Null:                   -4.1440e+06
Covariance Type:             nonrobust   LLR p-value:                     1.000
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Tot_Population_ACS_14_18                       0.0143      0.001     14.800      0.000       0.012       0.016
pct_URBANIZED_AREA_POP_CEN_2010                0.1707    107.432      0.002      0.999    -210.391     210.733
pct_URBAN_CLUSTER_POP_CEN_2010                 0.0932     68.946      0.001      0.999    -135.039     135.226
pct_RURAL_POP_CEN_2010                        -0.2791    104.609     -0.003      0.998    -205.310     204.751
avg_Tot_Prns_in_HHD_ACS_14_18                 -0.0059      0.001     -5.937      0.000      -0.008      -0.004
pct_Vacant_Units_ACS_14_18                    -0.0598      0.001    -67.478      0.000      -0.062      -0.058
pct_Hispanic_ACS_14_18                         0.0287      0.005      5.261      0.000       0.018       0.039
pct_NH_White_alone_ACS_14_18                  -0.1095      0.007    -14.673      0.000      -0.124      -0.095
pct_NH_Blk_alone_ACS_14_18                    -0.1086      0.005    -22.299      0.000      -0.118      -0.099
pct_NH_AIAN_alone_ACS_14_18                   -0.5635      0.004   -149.638      0.000      -0.571      -0.556
pct_NH_Asian_alone_ACS_14_18                  -0.0006      0.002     -0.273      0.785      -0.005       0.004
pct_NH_NHOPI_alone_ACS_14_18                   0.0047      0.001      6.026      0.000       0.003       0.006
pct_NH_SOR_alone_ACS_14_18                     0.0071      0.001      9.014      0.000       0.006       0.009
pct_Othr_Lang_ACS_14_18                       -0.1617      0.002    -70.122      0.000      -0.166      -0.157
pct_ENG_VW_ACS_14_18                           0.0320      0.001     26.119      0.000       0.030       0.034
pct_Not_HS_Grad_ACS_14_18                     -0.0476      0.001    -39.308      0.000      -0.050      -0.045
pct_College_ACS_14_18                          0.0084      0.001      6.107      0.000       0.006       0.011
avg_Agg_HH_INC_ACS_14_18                       0.0322      0.002     19.388      0.000       0.029       0.035
pct_Prs_Blw_Pov_Lev_ACS_14_18                 -0.0148      0.001    -14.298      0.000      -0.017      -0.013
pct_PUB_ASST_INC_ACS_14_18                     0.0389      0.001     48.295      0.000       0.037       0.040
pct_Diff_HU_1yr_Ago_ACS_14_18                  0.0060      0.001      7.275      0.000       0.004       0.008
pct_Recent_Built_HU_ACS_14_18                 -0.0142      0.001    -18.239      0.000      -0.016      -0.013
avg_Agg_House_Value_ACS_14_18                  0.0231      0.001     17.887      0.000       0.021       0.026
Primary RUCA Code 2010                        -1.5216      0.081    -18.719      0.000      -1.681      -1.362
Secondary RUCA Code, 2010 (see errata)         1.5031      0.081     18.499      0.000       1.344       1.662
Tract Population, 2010                         0.0118      0.001     12.990      0.000       0.010       0.014
Land Area (sq

#### observations:
Some coefficients have very large p values and standard errors, eg.pct_URBANIZED_AREA_POP_CEN_2010 (percent of population in a block group living in urbanized areas), and pct_RURAL_POP_CEN_2010 (percent of population in a block group living in rural areas).


But according to previous correlation analysis these are correlated with broadband availability, so we should be able to obtain better coefficient estimates.

In [14]:
# np.max(df_conus_num['Primary RUCA Code 2010']) # good max is not 99

#### refit model with some redundant features removed:
* still for the binary dependent variable AnyProviderWith25/3 (1 if a block has at least 1 provider providing MaxAdDown >= 25 and MaxAdUp >= 3, according to FCC form 477, 0 otherwise)

In [15]:
# remove redundant features
x2=df_conus_num[['Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
        'pct_RURAL_POP_CEN_2010','avg_Tot_Prns_in_HHD_ACS_14_18',
       'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18','Primary RUCA Code 2010', 'Tract Population, 2010',
       'Land Area (square miles), 2010',
       'Population Density (per square mile), 2010']]
# removed: 'Secondary RUCA Code, 2010 (see errata)', 'pct_URBAN_CLUSTER_POP_CEN_2010', 
# 'pct_PUB_ASST_INC_ACS_14_18', 'pct_NH_SOR_alone_ACS_14_18',


In [16]:
x2 = scaler.fit_transform(x2)
print(x2.mean(axis=0))
print(x2.std(axis=0))

[ 7.14510725e-18  9.41190616e-16 -1.28418303e-15  3.57271725e-15
 -2.52533180e-17 -7.24192072e-16  2.02893774e-15  3.26165964e-17
  8.46777023e-18  2.95267466e-16 -1.27827878e-16 -8.67448745e-16
  2.69495992e-16  1.56548754e-15  2.70022330e-15 -1.17812455e-16
 -4.39205924e-16  5.62199946e-16 -3.29438533e-16 -2.48442470e-17
  1.12030918e-16 -1.07312966e-16 -3.46892230e-16 -7.71944297e-16]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [17]:
x2=pd.DataFrame(x2,columns=['Tot_Population_ACS_14_18', 'pct_URBANIZED_AREA_POP_CEN_2010',
        'pct_RURAL_POP_CEN_2010','avg_Tot_Prns_in_HHD_ACS_14_18',
       'pct_Vacant_Units_ACS_14_18',
       'pct_Hispanic_ACS_14_18', 'pct_NH_White_alone_ACS_14_18',
       'pct_NH_Blk_alone_ACS_14_18', 'pct_NH_AIAN_alone_ACS_14_18',
       'pct_NH_Asian_alone_ACS_14_18', 'pct_NH_NHOPI_alone_ACS_14_18',
       'pct_Othr_Lang_ACS_14_18',
       'pct_ENG_VW_ACS_14_18', 'pct_Not_HS_Grad_ACS_14_18',
       'pct_College_ACS_14_18', 'avg_Agg_HH_INC_ACS_14_18',
       'pct_Prs_Blw_Pov_Lev_ACS_14_18',
       'pct_Diff_HU_1yr_Ago_ACS_14_18', 'pct_Recent_Built_HU_ACS_14_18',
       'avg_Agg_House_Value_ACS_14_18','Primary RUCA Code 2010', 'Tract Population, 2010',
       'Land Area (square miles), 2010',
       'Population Density (per square mile), 2010'])
x2

,Tot_Population_ACS_14_18,pct_URBANIZED_AREA_POP_CEN_2010,pct_RURAL_POP_CEN_2010,avg_Tot_Prns_in_HHD_ACS_14_18,pct_Vacant_Units_ACS_14_18,pct_Hispanic_ACS_14_18,pct_NH_White_alone_ACS_14_18,pct_NH_Blk_alone_ACS_14_18,pct_NH_AIAN_alone_ACS_14_18,pct_NH_Asian_alone_ACS_14_18,...,pct_College_ACS_14_18,avg_Agg_HH_INC_ACS_14_18,pct_Prs_Blw_Pov_Lev_ACS_14_18,pct_Diff_HU_1yr_Ago_ACS_14_18,pct_Recent_Built_HU_ACS_14_18,avg_Agg_House_Value_ACS_14_18,Primary RUCA Code 2010,"Tract Population, 2010","Land Area (square miles), 2010","Population Density (per square mile), 2010"
0,-0.714121,0.738754,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,-0.377860,...,-0.352976,-0.818769,0.245343,-1.017627,-0.436151,-0.550049,-0.818349,-1.221334,-0.337507,-0.300638
1,-0.714121,0.738754,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,-0.377860,...,-0.352976,-0.818769,0.245343,-1.017627,-0.436151,-0.550049,-0.818349,-1.221334,-0.337507,-0.300638
2,-0.714121,0.738754,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,-0.377860,...,-0.352976,-0.818769,0.245343,-1.017627,-0.436151,-0.550049,-0.818349,-1.221334,-0.337507,-0.300638
3,-0.714121,0.738754,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,-0.377860,...,-0.352976,-0.818769,0.245343,-1.017627,-0.436151,-0.550049,-0.818349,-1.221334,-0.337507,-0.300638
4,-0.714121,0.738754,-0.476168,-0.883283,-1.075868,-0.505450,0.359087,-0.074275,-0.180409,-0.377860,...,-0.352976,-0.818769,0.245343,-1.017627,-0.436151,-0.550049,-0.818349,-1.221334,-0.337507,-0.300638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8337435,-0.628434,-0.844677,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,-0.252091,...,-0.542532,-0.760154,0.605055,-0.071693,-0.229007,0.292540,1.914737,-1.439816,3.222259,-0.436382
8337436,-0.628434,-0.844677,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,-0.252091,...,-0.542532,-0.760154,0.605055,-0.071693,-0.229007,0.292540,1.914737,-1.439816,3.222259,-0.436382
8337437,-0.628434,-0.844677,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,-0.252091,...,-0.542532,-0.760154,0.605055,-0.071693,-0.229007,0.292540,1.914737,-1.439816,3.222259,-0.436382
8337438,-0.628434,-0.844677,1.149982,-1.043047,1.810788,-0.502791,0.758379,-0.469294,-0.027381,-0.252091,...,-0.542532,-0.760154,0.605055,-0.071693,-0.229007,0.292540,1.914737,-1.439816,3.222259,-0.436382


In [18]:
model2 = sm.Logit(y1, x2) 
result2 = model2.fit_regularized()
result2.summary()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.630771951049333
            Iterations: 97
            Function evaluations: 98
            Gradient evaluations: 97


<class 'statsmodels.iolib.summary.Summary'>
"""
                            Logit Regression Results                           
===============================================================================
Dep. Variable:     AnyProviderWith25/3   No. Observations:              8337440
Model:                           Logit   Df Residuals:                  8337416
Method:                            MLE   Df Model:                           23
Date:                 Wed, 19 May 2021   Pseudo R-squ.:                 -0.2691
Time:                         14:55:30   Log-Likelihood:            -5.2590e+06
converged:                        True   LL-Null:                   -4.1440e+06
Covariance Type:             nonrobust   LLR p-value:                     1.000
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Tot_Population_ACS_14_18                       0.0146      0.001     15.165      0.000       0.013       0.017
pct_URBANIZED_AREA_POP_CEN_2010                0.0261      0.001     18.459      0.000       0.023       0.029
pct_RURAL_POP_CEN_2010                        -0.4242      0.001   -321.579      0.000      -0.427      -0.422
avg_Tot_Prns_in_HHD_ACS_14_18                 -0.0038      0.001     -3.769      0.000      -0.006      -0.002
pct_Vacant_Units_ACS_14_18                    -0.0603      0.001    -68.085      0.000      -0.062      -0.059
pct_Hispanic_ACS_14_18                         0.0011      0.005      0.218      0.827      -0.009       0.011
pct_NH_White_alone_ACS_14_18                  -0.1490      0.007    -20.951      0.000      -0.163      -0.135
pct_NH_Blk_alone_ACS_14_18                    -0.1346      0.005    -28.907      0.000      -0.144      -0.125
pct_NH_AIAN_alone_ACS_14_18                   -0.5751      0.004   -156.717      0.000      -0.582      -0.568
pct_NH_Asian_alone_ACS_14_18                  -0.0091      0.002     -4.545      0.000      -0.013      -0.005
pct_NH_NHOPI_alone_ACS_14_18                   0.0043      0.001      5.536      0.000       0.003       0.006
pct_Othr_Lang_ACS_14_18                       -0.1630      0.002    -70.927      0.000      -0.168      -0.159
pct_ENG_VW_ACS_14_18                           0.0320      0.001     26.075      0.000       0.030       0.034
pct_Not_HS_Grad_ACS_14_18                     -0.0470      0.001    -38.827      0.000      -0.049      -0.045
pct_College_ACS_14_18                          0.0027      0.001      1.944      0.052   -2.19e-05       0.005
avg_Agg_HH_INC_ACS_14_18                       0.0301      0.002     18.151      0.000       0.027       0.033
pct_Prs_Blw_Pov_Lev_ACS_14_18                 -0.0068      0.001     -6.618      0.000      -0.009      -0.005
pct_Diff_HU_1yr_Ago_ACS_14_18                  0.0066      0.001      8.053      0.000       0.005       0.008
pct_Recent_Built_HU_ACS_14_18                 -0.0149      0.001    -19.215      0.000      -0.016      -0.013
avg_Agg_House_Value_ACS_14_18                  0.0243      0.001     18.791      0.000       0.022       0.027
Primary RUCA Code 2010                        -0.0179      0.001    -15.860      0.000      -0.020      -0.016
Tract Population, 2010                         0.0112      0.001     12.291      0.000       0.009       0.013
Land Area (square miles), 2010                -0.3725      0.002   -236.600      0.000      -0.376      -0.369
Population Density (per square mile), 2010     0.0575      0.001     56.238      0.000       0.055       0.059
==============================================================================================================
"""

sort coefficients in ascending order:

In [19]:
result2.params.sort_values()

pct_NH_AIAN_alone_ACS_14_18                  -0.575127
pct_RURAL_POP_CEN_2010                       -0.424235
Land Area (square miles), 2010               -0.372468
pct_Othr_Lang_ACS_14_18                      -0.163047
pct_NH_White_alone_ACS_14_18                 -0.149003
pct_NH_Blk_alone_ACS_14_18                   -0.134585
pct_Vacant_Units_ACS_14_18                   -0.060301
pct_Not_HS_Grad_ACS_14_18                    -0.046986
Primary RUCA Code 2010                       -0.017893
pct_Recent_Built_HU_ACS_14_18                -0.014944
pct_NH_Asian_alone_ACS_14_18                 -0.009090
pct_Prs_Blw_Pov_Lev_ACS_14_18                -0.006759
avg_Tot_Prns_in_HHD_ACS_14_18                -0.003770
pct_Hispanic_ACS_14_18                        0.001134
pct_College_ACS_14_18                         0.002655
pct_NH_NHOPI_alone_ACS_14_18                  0.004277
pct_Diff_HU_1yr_Ago_ACS_14_18                 0.006623
Tract Population, 2010                        0.011201
Tot_Popula

#### observations:
* overall lower p values and standard errors. The only features with p value > 0.05 are pct_Hispanic_ACS_14_18 (percentage of block group population identifying as hispanic) and pct_College_ACS_14_18 (percentage of block group population with college degree or above)
* Variable with the highest coefficient (ie. largest positive impact on probability of a block having at least 1 provider whose max advertised speeds meet 25/3 requirement) is Population Density (per square mile), 2010 (tract population density from RUCA data), this makes sense. 
* Variable with the second highest coefficient, pct_ENG_VW_ACS_14_18 (percentage of all ACS occupied housing units where no one ages14 years and over speaks English only or speaks English "very well"), this is interesting
* pct_NH_AIAN_alone_ACS_14_18 (percentage ofthe ACSpopulation that indicate no Hispanic originand their only race as "AmericanIndianor Alaska Native") has the lowest coefficient.

Check the confusion matrix and accuracy:

In [20]:
result2.pred_table()
# pred_table[i,j] refers to the number of times “i” was observed and the model predicted “j”. Correct predictions are along the diagonal.

array([[1354666.,  292658.],
       [2402478., 4287638.]])

In [21]:
(4287638+1354666)/(4287638+1354666+292658+2402478) # 67.6 accuracy

0.6767429810589342

* repeat for the binary dependent variable **AnyProviderWith100/10** (1 if a block has at least 1 provider providing MaxAdDown >= 100 and MaxAdUp >= 10, according to FCC form 477, 0 otherwise)

In [22]:
y2=df_conus_num['AnyProviderWith100/10']
y2.reset_index(drop=True,inplace=True)
y2

0          1
1          1
2          1
3          1
4          1
          ..
8337435    0
8337436    0
8337437    0
8337438    0
8337439    0
Name: AnyProviderWith100/10, Length: 8337440, dtype: int64

In [23]:
model3 = sm.Logit(y2, x2) 
result3 = model3.fit_regularized()
result3.summary()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5624150595000201
            Iterations: 83
            Function evaluations: 83
            Gradient evaluations: 83


<class 'statsmodels.iolib.summary.Summary'>
"""
                             Logit Regression Results                            
=================================================================================
Dep. Variable:     AnyProviderWith100/10   No. Observations:              8337440
Model:                             Logit   Df Residuals:                  8337416
Method:                              MLE   Df Model:                           23
Date:                   Wed, 19 May 2021   Pseudo R-squ.:                  0.1036
Time:                           15:10:11   Log-Likelihood:            -4.6891e+06
converged:                          True   LL-Null:                   -5.2311e+06
Covariance Type:               nonrobust   LLR p-value:                     0.000
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Tot_Population_ACS_14_18                       0.0195      0.001     18.512      0.000       0.017       0.022
pct_URBANIZED_AREA_POP_CEN_2010                0.1173      0.002     77.940      0.000       0.114       0.120
pct_RURAL_POP_CEN_2010                        -0.7210      0.001   -518.104      0.000      -0.724      -0.718
avg_Tot_Prns_in_HHD_ACS_14_18                 -0.0122      0.001    -11.234      0.000      -0.014      -0.010
pct_Vacant_Units_ACS_14_18                    -0.0243      0.001    -25.368      0.000      -0.026      -0.022
pct_Hispanic_ACS_14_18                        -0.0633      0.006    -11.281      0.000      -0.074      -0.052
pct_NH_White_alone_ACS_14_18                   0.0535      0.008      6.964      0.000       0.038       0.069
pct_NH_Blk_alone_ACS_14_18                    -0.0117      0.005     -2.328      0.020      -0.022      -0.002
pct_NH_AIAN_alone_ACS_14_18                   -0.3841      0.004   -103.452      0.000      -0.391      -0.377
pct_NH_Asian_alone_ACS_14_18                   0.0138      0.002      6.309      0.000       0.009       0.018
pct_NH_NHOPI_alone_ACS_14_18                  -0.0015      0.001     -1.780      0.075      -0.003       0.000
pct_Othr_Lang_ACS_14_18                       -0.0773      0.002    -30.957      0.000      -0.082      -0.072
pct_ENG_VW_ACS_14_18                          -0.0030      0.001     -2.308      0.021      -0.006      -0.000
pct_Not_HS_Grad_ACS_14_18                     -0.0126      0.001     -9.682      0.000      -0.015      -0.010
pct_College_ACS_14_18                          0.0525      0.001     35.127      0.000       0.050       0.055
avg_Agg_HH_INC_ACS_14_18                       0.0100      0.002      5.507      0.000       0.006       0.014
pct_Prs_Blw_Pov_Lev_ACS_14_18                  0.0256      0.001     23.283      0.000       0.023       0.028
pct_Diff_HU_1yr_Ago_ACS_14_18                 -0.0230      0.001    -25.956      0.000      -0.025      -0.021
pct_Recent_Built_HU_ACS_14_18                 -0.0366      0.001    -44.276      0.000      -0.038      -0.035
avg_Agg_House_Value_ACS_14_18                  0.0193      0.001     13.537      0.000       0.017       0.022
Primary RUCA Code 2010                        -0.0537      0.001    -43.936      0.000      -0.056      -0.051
Tract Population, 2010                         0.0253      0.001     25.762      0.000       0.023       0.027
Land Area (square miles), 2010                -0.4817      0.002   -241.657      0.000      -0.486      -0.478
Population Density (per square mile), 2010     0.1784      0.001    125.408      0.000       0.176       0.181
==============================================================================================================
"""

In [24]:
result3.params.sort_values()

pct_RURAL_POP_CEN_2010                       -0.720952
Land Area (square miles), 2010               -0.481686
pct_NH_AIAN_alone_ACS_14_18                  -0.384060
pct_Othr_Lang_ACS_14_18                      -0.077261
pct_Hispanic_ACS_14_18                       -0.063329
Primary RUCA Code 2010                       -0.053669
pct_Recent_Built_HU_ACS_14_18                -0.036601
pct_Vacant_Units_ACS_14_18                   -0.024274
pct_Diff_HU_1yr_Ago_ACS_14_18                -0.023033
pct_Not_HS_Grad_ACS_14_18                    -0.012560
avg_Tot_Prns_in_HHD_ACS_14_18                -0.012193
pct_NH_Blk_alone_ACS_14_18                   -0.011708
pct_ENG_VW_ACS_14_18                         -0.003038
pct_NH_NHOPI_alone_ACS_14_18                 -0.001455
avg_Agg_HH_INC_ACS_14_18                      0.010041
pct_NH_Asian_alone_ACS_14_18                  0.013769
avg_Agg_House_Value_ACS_14_18                 0.019346
Tot_Population_ACS_14_18                      0.019538
Tract Popu

Coefficient results are somewhat different from before

In [26]:
result3.pred_table()

array([[2195421.,  478679.],
       [1693119., 3970221.]])

In [27]:
(2195421+3970221)/(2195421+3970221+478679+1693119) #73.9 accuracy. better than 25/3

0.7395126081866856

In [69]:
# # multinomial regression on NumFCCEntriesWith25/3Speed?
# np.unique(df_conus_num['NumFCCEntriesWith25/3Speed'],return_counts=True) 

In [70]:
# np.sum(np.unique(df_conus_num['NumFCCEntriesWith25/3Speed'],return_counts=True)[1][5:])
# # make 5 & above a group

In [71]:
# np.unique(df_conus_num['NumFCCEntriesWith100/10Speed'],return_counts=True)

### Next steps:
1. Analyze urban and rural separately
2. Incorporate more geographical variables (such as tree covering, temperature, etc.)
